Classification of tasks using EEG data <br>
Using TensorFlow <br>
https://www.kaggle.com/wpncrh/classifying-tasks-using-eeg-data-w-tensorflow-nn

In [1]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('eeg-data.csv')

In [3]:
df['eeg_power'] = df.eeg_power.map(json.loads)

In [4]:
df.head(2)

,Unnamed: 0,id,indra_time,browser_latency,reading_time,attention_esense,meditation_esense,eeg_power,raw_values,signal_quality,createdAt,updatedAt,label
0,3730,12,2015-05-09 23:13:42.281,1461,2015-05-09 16:13:40.954,0,0,"[944412.0, 111373.0, 52404.0, 28390.0, 3237.0,...","[-203.0, -202.0, -196.0, -185.0, -163.0, -137....",200,2015-05-09 23:13:39.550,2015-05-09 23:13:39.549+00,unlabeled
1,3732,12,2015-05-09 23:13:43.186,1461,2015-05-09 16:13:41.964,0,0,"[1793049.0, 89551.0, 3896.0, 21727.0, 9301.0, ...","[104.0, 134.0, 128.0, 121.0, 145.0, 151.0, 123...",200,2015-05-09 23:13:40.559,2015-05-09 23:13:40.559+00,unlabeled


In [5]:
df.describe()

,Unnamed: 0,id,attention_esense,meditation_esense,signal_quality
count,30013.000000,30013.000000,30013.000000,30013.000000,30013.000000
mean,15010.952521,13.685570,40.284710,46.638723,25.871856
std,8666.911131,8.479506,24.446432,26.153089,64.636640
min,0.000000,1.000000,0.000000,0.000000,0.000000
25%,7506.000000,6.000000,24.000000,34.000000,0.000000
50%,15010.000000,13.000000,43.000000,50.000000,0.000000
75%,22517.000000,20.000000,57.000000,64.000000,0.000000
max,30021.000000,30.000000,100.000000,100.000000,200.000000


In [6]:
df = df.drop('Unnamed: 0', 1)
df = df.drop('indra_time', 1)
df = df.drop('browser_latency', 1)
df = df.drop('reading_time', 1)
df = df.drop('attention_esense', 1)
df = df.drop('meditation_esense', 1)
df = df.drop('raw_values', 1)
df = df.drop('signal_quality', 1)
df = df.drop('createdAt', 1)
df = df.drop('updatedAt', 1)

In [7]:
df.head()

,id,eeg_power,label
0,12,"[944412.0, 111373.0, 52404.0, 28390.0, 3237.0,...",unlabeled
1,12,"[1793049.0, 89551.0, 3896.0, 21727.0, 9301.0, ...",unlabeled
2,12,"[400192.0, 640624.0, 153087.0, 69733.0, 98854....",unlabeled
3,12,"[681192.0, 138630.0, 67891.0, 26459.0, 592240....",unlabeled
4,12,"[268406.0, 197772.0, 190654.0, 266433.0, 91683...",unlabeled


In [8]:
len(df)

30013

In [9]:
# seperate eeg power into multiple columns
to_series = pd.Series(df['eeg_power'])
eeg_features = pd.DataFrame(to_series.tolist())
df_eeg = pd.concat([df, eeg_features], axis=1)

In [10]:
df_eeg = df_eeg.drop('eeg_power', 1)

In [11]:
df_eeg.head()

,id,label,0,1,2,3,4,5,6,7
0,12,unlabeled,944412.0,111373.0,52404.0,28390.0,3237.0,32728.0,4845.0,2036.0
1,12,unlabeled,1793049.0,89551.0,3896.0,21727.0,9301.0,16096.0,3496.0,643.0
2,12,unlabeled,400192.0,640624.0,153087.0,69733.0,98854.0,199537.0,66993.0,51772.0
3,12,unlabeled,681192.0,138630.0,67891.0,26459.0,592240.0,171435.0,164399.0,41765.0
4,12,unlabeled,268406.0,197772.0,190654.0,266433.0,91683.0,200452.0,107585.0,57841.0


In [12]:
df_eeg['label'].value_counts()

unlabeled                       20054
relax                             934
music                             932
thinkOfItems-ver2                 481
video-ver2                        481
colorInstruction2                 469
video-ver1                        457
thinkOfItems-ver1                 449
thinkOfItemsInstruction-ver2      321
colorInstruction1                 316
thinkOfItemsInstruction-ver1      303
videoInstruction                  161
mathInstruction                   161
relaxInstruction                  158
musicInstruction                  152
blinkInstruction                  132
colorRound1-3                     128
colorRound1-4                     127
colorRound1-2                     125
colorRound1-1                     125
colorRound1-5                     124
colorRound1-6                     122
readyRound5                        96
colorRound5-3                      96
colorRound2-4                      95
readyRound4                        95
readyRound2 

In [13]:
df_eeg.dtypes

id         int64
label     object
0        float64
1        float64
2        float64
3        float64
4        float64
5        float64
6        float64
7        float64
dtype: object

In [14]:
df_eeg_ch = df_eeg.drop('id', 1)

In [15]:
# clean labels
def clean_labels(dd):
    dd.loc[dd.label == 'math1', 'label'] = "math"
    dd.loc[dd.label == 'math2', 'label'] = "math"
    dd.loc[dd.label == 'math3', 'label'] = "math"
    dd.loc[dd.label == 'math4', 'label'] = "math"
    dd.loc[dd.label == 'math5', 'label'] = "math"
    dd.loc[dd.label == 'math6', 'label'] = "math"
    dd.loc[dd.label == 'math7', 'label'] = "math"
    dd.loc[dd.label == 'math8', 'label'] = "math"
    dd.loc[dd.label == 'math9', 'label'] = "math"
    dd.loc[dd.label == 'math10', 'label'] = "math"
    dd.loc[dd.label == 'math11', 'label'] = "math"
    dd.loc[dd.label == 'math12', 'label'] = "math"
    dd.loc[dd.label == 'colorRound1-1', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound1-2', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound1-3', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound1-4', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound1-5', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound1-6', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound2-1', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound2-2', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound2-3', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound2-4', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound2-5', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound2-6', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound3-1', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound3-2', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound3-3', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound3-4', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound3-5', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound3-6', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound4-1', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound4-2', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound4-3', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound4-4', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound4-5', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound4-6', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound5-1', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound5-2', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound5-3', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound5-4', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound5-5', 'label'] = "colors"
    dd.loc[dd.label == 'colorRound5-6', 'label'] = "colors"
    dd.loc[dd.label == 'readyRound1', 'label'] = "ready"
    dd.loc[dd.label == 'readyRound2', 'label'] = "ready"
    dd.loc[dd.label == 'readyRound3', 'label'] = "ready"
    dd.loc[dd.label == 'readyRound4', 'label'] = "ready"
    dd.loc[dd.label == 'readyRound5', 'label'] = "ready"
    dd.loc[dd.label == 'video-ver1', 'label'] = "video"
    dd.loc[dd.label == 'video-ver2', 'label'] = "video"
    dd.loc[dd.label == 'blink1', 'label'] = "blink"
    dd.loc[dd.label == 'blink2', 'label'] = "blink"
    dd.loc[dd.label == 'blink3', 'label'] = "blink"
    dd.loc[dd.label == 'blink4', 'label'] = "blink"
    dd.loc[dd.label == 'blink5', 'label'] = "blink"
    dd.loc[dd.label == 'thinkOfItemsInstruction-ver1', 'label'] = "instruction"
    dd.loc[dd.label == 'thinkOfItemsInstruction-ver2', 'label'] = "instruction"
    dd.loc[dd.label == 'thinkOfItems-ver1', 'label'] = "instruction"
    dd.loc[dd.label == 'thinkOfItems-ver2', 'label'] = "instruction"
    dd.loc[dd.label == 'colorInstruction2', 'label'] = "instruction"
    dd.loc[dd.label == 'colorInstruction1', 'label'] = "instruction"
    dd.loc[dd.label == 'colorInstruction2', 'label'] = "instruction"
    dd.loc[dd.label == 'musicInstruction', 'label'] = "instruction"
    dd.loc[dd.label == 'videoInstruction', 'label'] = "instruction"
    dd.loc[dd.label == 'mathInstruction', 'label'] = "instruction"
    dd.loc[dd.label == 'relaxInstruction', 'label'] = "instruction"
    dd.loc[dd.label == 'blinkInstruction', 'label'] = "instruction"
    return dd

In [58]:
clean_labels_df = clean_labels(df_eeg_ch)

In [59]:
df_eeg_ch['label'].value_counts()

unlabeled      20054
instruction     3103
colors          2405
video            938
math             936
relax            934
music            932
ready            472
blink            239
Name: label, dtype: int64

In [167]:
# remove unwanted labels
df_eeg_chlabel = df_eeg_ch[df_eeg_ch['label'] != 'unlabeled']
df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'instruction']
df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'blink']
df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'ready']
df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'colors']
#df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'video']
df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'music']
df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'math']
#df_eeg_chlabel = df_eeg_chlabel[df_eeg_ch['label'] != 'relax']

/Users/dannie/anaconda3/envs/playtime/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Users/dannie/anaconda3/envs/playtime/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/dannie/anaconda3/envs/playtime/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/dannie/anaconda3/envs/playtime/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/dannie/anaconda3/envs/playtime/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/dannie/anaconda3/envs/playtime/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reinde

In [168]:
df_eeg_chlabel['label'].value_counts()

video    938
relax    934
Name: label, dtype: int64

In [169]:
# pop off labels to new group
label = df_eeg_chlabel.pop('label')
label.head()

13274    relax
13275    relax
13276    relax
13277    relax
13278    relax
Name: label, dtype: object

In [170]:
df_eeg_chlabel.head()

,0,1,2,3,4,5,6,7
13274,5044.0,10156.0,3281.0,10403.0,12393.0,10266.0,1949.0,2937.0
13275,548188.0,67192.0,20298.0,4142.0,30576.0,18237.0,5603.0,4783.0
13276,449571.0,83093.0,15379.0,34656.0,6750.0,10348.0,5315.0,2585.0
13277,85497.0,20547.0,2723.0,3270.0,2522.0,2209.0,449.0,393.0
13278,72768.0,44080.0,25974.0,16079.0,12995.0,27132.0,33264.0,9121.0


In [171]:
eeg_chlabel_scarray = StandardScaler().fit_transform(df_eeg_chlabel)
df_eeg_chlabel_sc = pd.DataFrame(eeg_chlabel_scarray)
df_eeg_chlabel_sc.head()

,0,1,2,3,4,5,6,7
0,-0.575080,-0.511652,-0.555411,-0.393464,-0.014575,-0.067708,-0.498317,-0.319119
1,0.577046,0.064760,-0.153283,-0.584632,1.277384,0.609695,-0.124305,-0.044454
2,0.367858,0.225457,-0.269524,0.347057,-0.415528,-0.060739,-0.153784,-0.371493
3,-0.404422,-0.406639,-0.568597,-0.611257,-0.715941,-0.752419,-0.651853,-0.697639
4,-0.431423,-0.168812,-0.019154,-0.220157,0.028199,1.365622,2.706987,0.600993


In [172]:
eeg_values = df_eeg_chlabel_sc.values

In [173]:
eeg_values

array([[-0.5750801 , -0.51165187, -0.55541057, ..., -0.06770789,
        -0.4983173 , -0.31911905],
       [ 0.57704636,  0.06476019, -0.15328334, ...,  0.60969476,
        -0.12430547, -0.04445433],
       [ 0.36785827,  0.22545743, -0.2695238 , ..., -0.06073925,
        -0.15378424, -0.37149282],
       ...,
       [-0.18664278, -0.3194943 , -0.42796906, ..., -0.79669546,
        -0.56904586, -0.02079686],
       [ 0.54841629, -0.13830187, -0.06242248, ...,  0.83362601,
         0.53466775,  0.63253619],
       [-0.57323464, -0.53194494, -0.62200248, ..., -0.53469165,
         1.58925009,  1.15791058]])

In [174]:
# convert labels to onehots
train_labels = pd.get_dummies(label)

In [175]:
train_labels.shape

(1872, 2)

In [176]:
# convert train_labels to np array
train_labels_values = train_labels.values

In [177]:
x_train, x_test, y_train, y_test = train_test_split(eeg_values,
                                                    train_labels_values,
                                                    test_size=0.2)

In [178]:
x_train, x_test, y_train, y_test = np.array(x_train, dtype='float32'), np.array(x_test, dtype='float32'), np.array(y_train, dtype='float32'), np.array(y_test, dtype='float32')

In [179]:
# build a simple tensorflow model with 1 hidden layer with 1000 nodes in this layer
x = tf.placeholder(tf.float32, [None, x_train.shape[1]])
w1 = tf.Variable(tf.random_normal([x_train.shape[1], 1000], stddev=.5, name='w1'))
b1 = tf.Variable(tf.zeros([1000]))
# calculate hidden output
hidden_output = tf.nn.softmax(tf.matmul(x, w1) + b1)
# bring from 1000 nodes to one of 8 possible labels
w2 = tf.Variable(tf.random_normal([1000, y_train.shape[1]], stddev=.5, name='w2'))
b2 = tf.Variable(tf.zeros([y_train.shape[1]]))
# placeholder for correct values
y_ = tf.placeholder('float', [None, y_train.shape[1]])
# implement model, find predicted y
y = tf.nn.softmax(tf.matmul(hidden_output, w2) + b2)

In [180]:
# use ADAM optimizer to adjust learning rate over time
loss = tf.reduce_mean(tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_, name='xentropy')))
opt = tf.train.AdamOptimizer(learning_rate=.0005)
train_step = opt.minimize(loss, var_list=[w1, b1, w2, b2])

In [181]:
# get mini_batch, so that we aren't feeding data in every training epoch
def get_mini_batch(x,y):
    rows = np.random.choice(x.shape[0], 100)
    return x[rows], y[rows]

In [182]:
sess = tf.Session()
# init all vars in graph
## init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
sess.run(init)

In [183]:
ntrials = 10000
for i in range(ntrials):
    # get mini batch
    a,b = get_mini_batch(x_train, y_train)
    # run train step, feeding arrays of 100 rows at a time
    _, cost = sess.run([train_step, loss], feed_dict={x: a, y_: b})
    if i%500 == 0:
        print('epoch is {0} and cost is {1}'.format(i, cost))
        
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

epoch is 0 and cost is 69.56936645507812
epoch is 500 and cost is 67.87948608398438
epoch is 1000 and cost is 68.73283386230469
epoch is 1500 and cost is 65.89002990722656
epoch is 2000 and cost is 63.916561126708984
epoch is 2500 and cost is 65.16968536376953
epoch is 3000 and cost is 65.02008819580078
epoch is 3500 and cost is 63.388633728027344
epoch is 4000 and cost is 64.01756286621094
epoch is 4500 and cost is 60.71466827392578
epoch is 5000 and cost is 61.37683868408203
epoch is 5500 and cost is 62.655662536621094
epoch is 6000 and cost is 61.11294174194336
epoch is 6500 and cost is 61.66944122314453
epoch is 7000 and cost is 59.77870178222656
epoch is 7500 and cost is 62.694610595703125
epoch is 8000 and cost is 62.848548889160156
epoch is 8500 and cost is 62.31763458251953
epoch is 9000 and cost is 64.17024230957031
epoch is 9500 and cost is 61.29704284667969


In [184]:
print("test accuracy is {}".format(sess.run(accuracy, feed_dict={x: x_test, y_: y_test})))

test accuracy is 0.5893333554267883


In [130]:
sess.close

<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x125f05f60>>

music/relax > 0.51 <br>
math/relax > 0.63 <br>
math/music > 0.623 <br>
math/video > 0.57 <br>
music/video > 0.6096 <br>
video/relax > 0.589
